In [1]:
import pyperclip,re,pyautogui,time
import pandas as pd
from random import choice
import jieba.analyse
import time

pyautogui.PAUSE = 0.102

In [ ]:
urllist = pd.read_csv('listt.csv',header=None)[0].to_list()

In [ ]:
diyichuangkou = (146,9)
urlxy = (382,52)

name1 = (820,297)
name2 = (1325,296)

blankk = (300,400)
xiayiye0 = (1262,510)
xiayiye = (1262,550)

fw1 = (815,601)
fw2 = (1186,604)

def ppaste(astr):
    pyperclip.copy(astr)
    pyautogui.hotkey('ctrl', 'v')
    
def clickk(atuple):
    pyautogui.moveTo(atuple,duration=0.101,pause=0.101)
    pyautogui.click(pause=0.101)

def loc(apng):
    try:
        xiayiye = pyautogui.locateCenterOnScreen(apng,grayscale=False)
        return xiayiye
    except:
        return None

def find_nextpage():
    pyautogui.hotkey('ctrl','f',pause=0.15)
    pyautogui.press('enter',pause=0.15)
    pyautogui.press('esc',pause=0.15)
    clickk(blankk)

def find_from_source(asource):
    try:
        asource = asource.replace('\r\n','')
        pinglun1 = pd.Series(re.findall(r'有内容按默.{10,10000}?（匿',asource,flags=re.DOTALL))
        pinglun1 = pinglun1.map(lambda x:x[6:])
        pinglun2 = pd.Series(re.findall(r'名）.{10,10000}?（匿',asource,flags=re.DOTALL))
        pinglun = pd.concat([pinglun1,pinglun2])        

        
        pinglun = pinglun.str.replace('.{5,5}解释：.*','')
        pinglun = pinglun.str.replace('.*天后追加：','')
        pinglun = pinglun.str.replace('超级会员|名）|.{5,5}（匿','')
        pinglun = pinglun.str.replace('[0-9][0-9].{1,1}[0-9][0-9]','')
        pinglun = pinglun.str.replace('：.*','')
        pinglun = pinglun.str.replace('颜色分类','')
        pinglun = pinglun[pinglun.map(lambda x: False if '下一页' in x else True)]
        pinglun = pinglun[pinglun.map(lambda x: False if '查看更多的' in x else True)]
        return pinglun
    except:
        return pd.Series()

In [ ]:
for i in urllist:
    df = pd.Series()
    skuurl = i
    assert 'detail.tmall' in skuurl
    
    #更改 URL 链接
    pyautogui.click(diyichuangkou,pause=0.5)
    pyautogui.click(urlxy,pause=0.5)
    pyautogui.hotkey('ctrl','a',pause=0.5)
    ppaste(skuurl)
    pyautogui.press('enter',pause=0.5)
    for m in range(15):
        pyautogui.press('pageup')
   
    #从页面中提取到商品名称，filename，washto-rightname
    pyautogui.click(name1,pause=0.5)
    pyautogui.dragTo(name2,duration=2)
    pyautogui.hotkey('ctrl','c')
    filename = pyperclip.paste()
    filename = filename.replace('*','').replace('\/','').replace('\'','')
    print(filename)
    
    #定位到商品详情，并点击到评论的页面
    pyautogui.press('pagedown')
    pyautogui.hotkey('ctrl','f')
    ppaste('商品详情')
    pyautogui.press('enter')
    pyautogui.press('esc')
    clickk(blankk)

    leijipingjia = loc('leijipingjia.png')
    pyautogui.click(leijipingjia,duration=2)
   
    #把搜索词更改为‘下一页’
    pyautogui.hotkey('ctrl','f')
    ppaste('下一页')
    pyautogui.press('enter')
    pyautogui.press('esc')
    pyautogui.click(blankk,duration=1)
    
    pyautogui.hotkey('ctrl','a',pause=1)
    pyautogui.hotkey('ctrl','c',pause=1)
    zhengwen = pyperclip.paste()
    pagenum = int(re.findall('累计评价 [0-9].*',zhengwen)[0][5:-1])
    pagehash = pagenum // 20
    pagehash = 100 if pagehash>100 else pagehash
    
    for c in range(15):
        pyautogui.press('pageup')
    
    for n in range(pagehash):
        clickk(blankk)
        pyautogui.hotkey('ctrl','a',pause=0.2)
        pyautogui.hotkey('ctrl','c',pause=0.2)
        zhengwen = pyperclip.paste()
    
        try:
            pinglun = find_from_source(zhengwen)
            df = pd.concat([df,pinglun])
        except:
            raise TypeError
    
        if '为什么被折叠' in zhengwen:
            break
        
        if pagenum < 20:
            break
        
        if '访问验证' in zhengwen:
            raise SyntaxError

        find_nextpage()
        clickk(xiayiye0) if i==0 else clickk(xiayiye)
        
    rizhipath = filename + '.xlsx'
    pd.DataFrame([]).to_excel(rizhipath)
    excelWriter = pd.ExcelWriter(rizhipath)
    
    df.drop_duplicates(inplace=True)
    df.to_excel(excelWriter,index=False,encoding='gb18030',sheet_name='源数据',header=None)
    gjc=[]
    for x in df.map(jieba.analyse.textrank):
        for y in x:
            gjc.append(y)
    pd.Series(gjc).value_counts().to_excel(excelWriter,sheet_name='源数据词频统计')
    
    excelWriter.save()
    excelWriter.close()
    pd.Series(urllist).iloc[n+1:].to_csv('listt.csv',header=None,index=False)

阿迪达斯 三叶草STAN SMITH男女经典鞋M20327M20326 
